# Flightdata Preprocessor

In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import logging
import matplotlib.pyplot as plt
import sys

In [352]:
sys.path.append('./')

In [351]:
df = pd.read_csv('../assets/data.txt', sep="\t", encoding="utf-8")
del df['nix']

In [171]:
df.head()

,Flight,Date,Origin,Destination,Equipment,Flight Time,ETD,ATD,ETA,ATA
0,LH922,18-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AING),1:09,21:30,21:42,22:10,Landed 21:50
1,LH922,17-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AINL),1:03,21:30,22:04,22:10,Landed 22:08
2,LH922,16-Oct-20,Frankfurt (FRA),London (LHR),A320 (D-AIUY),1:07,21:30,21:41,22:10,Landed 21:47
3,LH922,15-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AIJB),1:08,21:30,21:39,22:10,Landed 21:48
4,LH922,14-Oct-20,Frankfurt (FRA),London (LHR),A20N (D-AINJ),1:06,21:30,21:34,22:10,Landed 21:40


In [172]:
def clean_data(df):
    logging.warning(f'Received df of len {len(df)}...')
    for _ in ['Origin', 'Destination', 'Equipment']:
        df = df[df[_].str.contains("\)")]
    for _ in ['Flight Time', 'ETD', 'ATD', 'ETA', 'ATA']:
        df = df[~(df[_].str.contains("-")) & ~(df[_].str.contains("—"))]
    df = df[(df['ATA'].str.contains("Landed"))]
    df['ATA'] = df['ATA'].str.replace('Landed ','')
    df['ATA'] = df['ATA'].str.replace(u'Landed\xa0','')
    logging.warning(f'Returned df of len {len(df)}...')
    return df


In [173]:
_df = clean_data(df)

In [174]:
def clean_codes(df):
    _df = df.copy()
    for _ in ['Origin', 'Destination']:
        _df[[f'{_}_City', f'{_}_Code']] = _df[_].str.split(u"\xa0\(", n = 1, expand = True)
    _df[['Equipment_Type', 'Equipment_Reg']] = _df['Equipment'].str.split(u"\xa0\(", n = 1, expand = True)
    for _ in ['Equipment_Reg', 'Origin_Code', 'Destination_Code']:
        _df[_] = _df[_].str.replace(")", "")
    return _df[['Flight', 'Date', 'Equipment_Type', 'Equipment_Reg', 'Origin_City', 'Origin_Code', 'Destination_City', 'Destination_Code', 'Flight Time', 'ETD', 'ETA', 'ATD', 'ATA']]

In [175]:
_df = clean_codes(_df)

In [201]:
def get_carrier_code(df):
    _df = df.copy()
    carrier = {}
    carrier_list = []
    for index, row in _df.iterrows():
        if row['Flight'][:2] not in carrier.keys():
            carrier[row['Flight'][:2]] = ""
        carrier_list.append(row['Flight'][:2])
    return carrier, carrier_list

In [202]:
carrier, carrier_list = get_carrier_code(_df)

In [214]:
def set_carrier_info(df, carrier, carrier_class, carrier_list):
    _df = df.copy()
    _df['Carrier_Name'] = carrier_list
    _df['Carrier_Name'] = _df['Carrier_Name'].map(carrier)
    _df['Carrier_Class'] = _df['Carrier_Name'].map(carrier_class)
    return _df

In [218]:
_df = set_carrier_info(_df, carrier, carrier_class, carrier_list)

In [208]:
carrier = {
    'LH': 'Lufthansa',
    'BA': 'British Airways',
    'LO': 'LOT',
    'SK': 'SAS',
    'UX': 'Air Europa',
    'IB': 'Iberia',
    'KL': 'KLM Royal Dutch Airlines',
    'DY': 'Norwegian',
    'ET': 'Ethiopian',
    'FR': 'Ryanair',
    'HV': 'Transavia',
    'VY': 'Vueling',
    'EI': 'Aer Lingus',
    'U2': 'EasyJet'
    }

In [217]:
carrier_class = {
    'Lufthansa': 0,
    'British Airways': 0,
    'LOT': 0,
    'SAS': 0,
    'Air Europa': 0,
    'Iberia': 0,
    'KLM Royal Dutch Airlines': 0,
    'Norwegian': 1,
    'Ethiopian': 0,
    'Ryanair': 1,
    'Transavia': 1,
    'Vueling': 1,
    'Aer Lingus': 0,
    'EasyJet': 1
    }

In [227]:
def detect_dates(df):
    _df = df.copy()
    date_list = []
    for index, row in _df.iterrows():
        date_list.append(datetime.strptime(row['Date'], '%d-%b-%y').date().strftime('%Y-%m-%d'))
        
    _df['Date'] = date_list
    return _df

In [228]:
_df = detect_dates(_df)

In [324]:
def get_time_delta(df):
    _df = df.copy()
    deltas = []
    for index, row in _df.iterrows():
        tdelta = datetime.strptime(row['ATA'], '%H:%M') - datetime.strptime(row['ETA'], '%H:%M')
        #if tdelta.days < 0:
        #tdelta = timedelta(seconds = tdelta.seconds, microseconds = tdelta.microseconds)
        if datetime.strptime(row['ATA'], '%H:%M') < datetime.strptime(row['ETA'], '%H:%M'):
            deltas.append(-1 * int((datetime.strptime('00:00', '%H:%M') - tdelta).time().minute))
        else:
            deltas.append(int((datetime.strptime('00:00', '%H:%M') + tdelta).time().minute))

    _df['Timedelta'] = deltas
    return _df

In [325]:
_df = get_time_delta(_df)

In [333]:
def get_departure_groups(df):
    _df = df.copy()
    dep_groups = []
    for index, row in _df.iterrows():
        if datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('07:30', '%H:%M'):
            dep_groups.append('Red Eye')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('12:00', '%H:%M'):
            dep_groups.append('Morning')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('18:00', '%H:%M'):
            dep_groups.append('Afternoon')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('21:30', '%H:%M'):
            dep_groups.append('Evening')
        elif datetime.strptime(row['ETD'], '%H:%M') <= datetime.strptime('23:59', '%H:%M'):
            dep_groups.append('Night')
        else:
            dep_groups.append(None)
            logging.warning('invalid time?')
    _df['ETD_Group'] = dep_groups
    return _df

In [335]:
_df = get_departure_groups(_df)

In [66]:
def vote_majority(df):
    _df = pd.DataFrame()
    df_filter = df.groupby(['Flight'], as_index=False).max()[['Flight', 'Origin_City', 'Destination_City']]
    for index, row in df_filter.iterrows():
        df_temp = df[(df['Flight'] == row['Flight']) & (df['Origin_City'] == row['Origin_City']) & (df['Destination_City'] == row['Destination_City'])]
        _df = _df.append(df_temp)
    return _df


In [67]:
_df = vote_majority(df)

In [68]:
_df.to_csv('../assets/data_prep_majority.txt', sep="\t", encoding="utf-8", index=False)

In [3]:
df = pd.read_csv('../assets/data_prep.txt', sep="\t", encoding="utf-8")